In [3]:
import nltk
from nltk.translate.bleu_score import sentence_bleu
from typing import List
import pandas as pd
import sacrebleu
from rouge_score import rouge_scorer
import evaluate
rouge = evaluate.load('rouge')
sacrebleu = evaluate.load("sacrebleu")
chrf = evaluate.load("chrf")

/Users/israela/anaconda3/envs/orca/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# d = pd.read_csv('./amharic_title_generation/gpt4_responses-amharic_title_generation.csv').input.values.tolist()


In [5]:

data = pd.read_csv('./Gpt-4/amharic_mt_eng-amh/')
print(data.datasource.value_counts())
data.head()

IsADirectoryError: [Errno 21] Is a directory: './Gpt-4/amharic_mt_eng-amh/'

In [ ]:
# data=data[data.Datasource=="amharic_mt amh-eng"]
# data.drop_duplicates(subset='Input Text', keep="last")
# data = data.head(1300)
data = data.fillna('')
data.tail()

NameError: name 'data' is not defined

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/israela/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [11]:
def calculate_bleu_score(references: List[List[str]], candidates: List[str]) -> float:
    scores = []
    for ref, cand in zip(references, candidates):
        candidate_tokens = nltk.word_tokenize(cand)
        reference_tokens = [nltk.word_tokenize(r) for r in ref]
        
        score = sentence_bleu(reference_tokens, candidate_tokens)
        scores.append(score)

    
    print(f" sentence bleu : {sum(scores) / len(scores) if scores else 0}")

    bleu = sacrebleu.compute(predictions=candidates, references=references)
    print(f"SacreBLEU score: {bleu}")

    chr = chrf.compute(predictions=candidates, references=references, word_order=2)
    print(f"corpus_chrf score: {chr}")

    results = rouge.compute(predictions=candidates, references=references)
    print(f"Rogue results: {results}")

   





In [12]:
calculate_bleu_score(data.ouptput.values.tolist(), data.response.values.tolist())


/Users/israela/anaconda3/envs/orca/lib/python3.8/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/Users/israela/anaconda3/envs/orca/lib/python3.8/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/Users/israela/anaconda3/envs/orca/lib/python3.8/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider 

 sentence bleu : 9.531815028307571e-232
SacreBLEU score: {'score': 17.69852363762323, 'counts': [13367, 5993, 3261, 1864], 'totals': [28084, 27072, 26060, 25048], 'precisions': [47.59649622560889, 22.137263593380613, 12.513430544896393, 7.441711913126796], 'bp': 1.0, 'sys_len': 28084, 'ref_len': 24721}
corpus_chrf score: {'score': 47.0159345682244, 'char_order': 6, 'word_order': 0, 'beta': 2}
Rogue results: {'rouge1': 0.5086489691307752, 'rouge2': 0.2499554893717757, 'rougeL': 0.4336314740666024, 'rougeLsum': 0.4339841243367373}
